In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
import dotenv
import json
import os
import requests
import psycopg
import sqlite3
from sqlalchemy import create_engine

dotenv.load_dotenv()


True

In [2]:
ct = contrans()
ct.mypassword
ct.congresskey

'TEpTHpL0UEqkAiMhHerXxdMW9or5xGze4urGA00R'

In [3]:
members = ct.get_bioguideIDs()
terms, members = ct.terms_df(members)
members =  ct.make_cand_table(members)

In [4]:
members.head(3).T

,0,1,2
bioguideId,L000397,L000604,M001225
district,18,04,15
name,"Lofgren, Zoe","Lopez, Greg","Mullin, Kevin"
partyName,Democratic,Republican,Democratic
state,CA,CO,CA
updateDate,2024-10-22T14:54:19Z,2024-10-10T21:25:00Z,2024-10-10T21:22:27Z
url,https://api.congress.gov/v3/member/L000397?for...,https://api.congress.gov/v3/member/L000604?for...,https://api.congress.gov/v3/member/M001225?for...
depiction.attribution,Image courtesy of the Member,Image courtesy of the Member,Image courtesy of the Member
depiction.imageUrl,https://www.congress.gov/img/member/671024d7ec...,https://www.congress.gov/img/member/668e94fd65...,https://www.congress.gov/img/member/m001225_20...
partyletter,D,R,D


In [5]:
terms.head(3).T

,0,0,0
chamber,House of Representatives,House of Representatives,House of Representatives
startYear,1995,2024,2023
bioguideId,L000397,L000604,M001225
endYear,NaN,NaN,NaN


In [3]:
members =  ct.make_cand_table()
members

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item,partyletter,DistIDRunFor,lastname,firstname,name2,CID
0,L000397,18,"Lofgren, Zoe",Democratic,CA,2024-10-22T14:54:19Z,https://api.congress.gov/v3/member/L000397?for...,Image courtesy of the Member,https://www.congress.gov/img/member/671024d7ec...,"[{'chamber': 'House of Representatives', 'star...",D,CA18,Lofgren,Zoe,Lofgren (D),N00007479
1,L000604,04,"Lopez, Greg",Republican,CO,2024-10-10T21:25:00Z,https://api.congress.gov/v3/member/L000604?for...,Image courtesy of the Member,https://www.congress.gov/img/member/668e94fd65...,"[{'chamber': 'House of Representatives', 'star...",R,CO04,Lopez,Greg,Lopez (R),NaN
2,M001225,15,"Mullin, Kevin",Democratic,CA,2024-10-10T21:22:27Z,https://api.congress.gov/v3/member/M001225?for...,Image courtesy of the Member,https://www.congress.gov/img/member/m001225_20...,"[{'chamber': 'House of Representatives', 'star...",D,CA15,Mullin,Kevin,Mullin (D),N00049438
3,M001229,10,"McIver, LaMonica",Democratic,NJ,2024-10-10T21:19:13Z,https://api.congress.gov/v3/member/M001229?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66fd489d79...,"[{'chamber': 'House of Representatives', 'star...",D,NJ10,McIver,LaMonica,McIver (D),NaN
4,H001097,S,"Helmy, George S.",Democratic,NJ,2024-10-10T12:42:18Z,https://api.congress.gov/v3/member/H001097?for...,Official U.S. Senate Photo,https://www.congress.gov/img/member/6705445b99...,"[{'chamber': 'Senate', 'startYear': 2024}]",D,NJS,Helmy,George S.,Helmy (D),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,B001267,S,"Bennet, Michael F.",Democratic,CO,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001267?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001267_20...,"[{'chamber': 'Senate', 'startYear': 2009}]",D,COS,Bennet,Michael F.,Bennet (D),N00030608
535,B001243,S,"Blackburn, Marsha",Republican,TN,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001243?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001243_20...,"[{'chamber': 'House of Representatives', 'endY...",R,TNS,Blackburn,Marsha,Blackburn (R),N00003105
536,B001236,S,"Boozman, John",Republican,AR,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001236?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001236_20...,"[{'chamber': 'House of Representatives', 'endY...",R,ARS,Boozman,John,Boozman (R),N00013873
537,B001230,S,"Baldwin, Tammy",Democratic,WI,2024-03-10T12:42:12Z,https://api.congress.gov/v3/member/B001230?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001230_20...,"[{'chamber': 'House of Representatives', 'endY...",D,WIS,Baldwin,Tammy,Baldwin (D),N00004367


In [4]:
terms, members = ct.terms_df(members)

In [5]:
terms

,chamber,startYear,bioguideId,endYear
0,House of Representatives,1995,L000397,NaN
0,House of Representatives,2024,L000604,NaN
0,House of Representatives,2023,M001225,NaN
0,House of Representatives,2024,M001229,NaN
0,Senate,2024,H001097,NaN
...,...,...,...,...
0,House of Representatives,2001,B001236,2011.0
1,Senate,2011,B001236,NaN
0,House of Representatives,1999,B001230,2013.0
1,Senate,2013,B001230,NaN


In [6]:
members

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,partyletter,DistIDRunFor,lastname,firstname,name2,CID
0,L000397,18,"Lofgren, Zoe",Democratic,CA,2024-10-22T14:54:19Z,https://api.congress.gov/v3/member/L000397?for...,Image courtesy of the Member,https://www.congress.gov/img/member/671024d7ec...,D,CA18,Lofgren,Zoe,Lofgren (D),N00007479
1,L000604,04,"Lopez, Greg",Republican,CO,2024-10-10T21:25:00Z,https://api.congress.gov/v3/member/L000604?for...,Image courtesy of the Member,https://www.congress.gov/img/member/668e94fd65...,R,CO04,Lopez,Greg,Lopez (R),NaN
2,M001225,15,"Mullin, Kevin",Democratic,CA,2024-10-10T21:22:27Z,https://api.congress.gov/v3/member/M001225?for...,Image courtesy of the Member,https://www.congress.gov/img/member/m001225_20...,D,CA15,Mullin,Kevin,Mullin (D),N00049438
3,M001229,10,"McIver, LaMonica",Democratic,NJ,2024-10-10T21:19:13Z,https://api.congress.gov/v3/member/M001229?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66fd489d79...,D,NJ10,McIver,LaMonica,McIver (D),NaN
4,H001097,S,"Helmy, George S.",Democratic,NJ,2024-10-10T12:42:18Z,https://api.congress.gov/v3/member/H001097?for...,Official U.S. Senate Photo,https://www.congress.gov/img/member/6705445b99...,D,NJS,Helmy,George S.,Helmy (D),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,B001267,S,"Bennet, Michael F.",Democratic,CO,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001267?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001267_20...,D,COS,Bennet,Michael F.,Bennet (D),N00030608
535,B001243,S,"Blackburn, Marsha",Republican,TN,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001243?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001243_20...,R,TNS,Blackburn,Marsha,Blackburn (R),N00003105
536,B001236,S,"Boozman, John",Republican,AR,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001236?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001236_20...,R,ARS,Boozman,John,Boozman (R),N00013873
537,B001230,S,"Baldwin, Tammy",Democratic,WI,2024-03-10T12:42:12Z,https://api.congress.gov/v3/member/B001230?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001230_20...,D,WIS,Baldwin,Tammy,Baldwin (D),N00004367


In [10]:
members['terms.item'][3]

[{'chamber': 'House of Representatives', 'startYear': 2024}]

In [4]:
ideology = ct.get_ideology()
ideology

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,118,House,20301,41,3,AL,200,NaN,NaN,"ROGERS, Mike Dennis",...,NaN,0.379,0.376,-161.36951,0.85666,1043.0,79.0,NaN,0.449,0.348
1,118,House,21102,41,7,AL,100,NaN,NaN,"SEWELL, Terri",...,NaN,-0.402,0.396,-51.44477,0.94995,1002.0,18.0,NaN,-0.524,0.368
2,118,House,21500,41,6,AL,200,NaN,NaN,"PALMER, Gary James",...,NaN,0.677,0.109,-88.04471,0.91949,1049.0,42.0,NaN,0.674,0.141
3,118,House,22108,41,1,AL,200,NaN,NaN,"CARL, Jerry L.",...,NaN,0.550,0.237,-124.31414,0.88973,1064.0,56.0,NaN,0.599,0.236
4,118,House,22140,41,2,AL,200,NaN,NaN,"MOORE, Barry",...,NaN,0.641,-0.247,-98.75167,0.91121,1062.0,42.0,NaN,0.715,-0.298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,118,House,21970,25,1,WI,200,NaN,NaN,"STEIL, Bryan",...,NaN,0.417,0.062,-116.72081,0.89583,1061.0,45.0,NaN,0.437,0.130
445,118,House,21989,25,7,WI,200,NaN,NaN,"TIFFANY, Thomas P.",...,NaN,0.641,-0.250,-91.17689,0.91713,1054.0,35.0,NaN,0.665,-0.286
446,118,House,22115,25,5,WI,200,NaN,NaN,"FITZGERALD, Scott",...,NaN,0.599,0.084,-99.09669,0.91091,1062.0,47.0,NaN,0.585,-0.013
447,118,House,22370,25,3,WI,200,NaN,NaN,"VAN ORDEN, Derrick",...,NaN,0.377,-0.014,-186.77268,0.83503,1036.0,84.0,NaN,0.376,-0.012


In [5]:
votes = ct.get_votes()
votes

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,118,House,1,14854,6,100.0
1,118,House,1,14863,6,98.2
2,118,House,1,14873,1,100.0
3,118,House,1,15029,1,100.0
4,118,House,1,15433,1,100.0
...,...,...,...,...,...,...
510383,118,House,1173,31101,6,99.9
510384,118,House,1173,31102,1,99.6
510385,118,House,1173,39301,6,100.0
510386,118,House,1173,39307,6,100.0


In [9]:
members = ct.get_bioguideIDs()
members

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item
0,L000397,18.0,"Lofgren, Zoe",Democratic,California,2024-10-22T14:54:19Z,https://api.congress.gov/v3/member/L000397?for...,Image courtesy of the Member,https://www.congress.gov/img/member/671024d7ec...,"[{'chamber': 'House of Representatives', 'star..."
1,L000604,4.0,"Lopez, Greg",Republican,Colorado,2024-10-10T21:25:00Z,https://api.congress.gov/v3/member/L000604?for...,Image courtesy of the Member,https://www.congress.gov/img/member/668e94fd65...,"[{'chamber': 'House of Representatives', 'star..."
2,M001225,15.0,"Mullin, Kevin",Democratic,California,2024-10-10T21:22:27Z,https://api.congress.gov/v3/member/M001225?for...,Image courtesy of the Member,https://www.congress.gov/img/member/m001225_20...,"[{'chamber': 'House of Representatives', 'star..."
3,M001229,10.0,"McIver, LaMonica",Democratic,New Jersey,2024-10-10T21:19:13Z,https://api.congress.gov/v3/member/M001229?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66fd489d79...,"[{'chamber': 'House of Representatives', 'star..."
4,H001097,NaN,"Helmy, George S.",Democratic,New Jersey,2024-10-10T12:42:18Z,https://api.congress.gov/v3/member/H001097?for...,Official U.S. Senate Photo,https://www.congress.gov/img/member/6705445b99...,"[{'chamber': 'Senate', 'startYear': 2024}]"
...,...,...,...,...,...,...,...,...,...,...
34,B001267,NaN,"Bennet, Michael F.",Democratic,Colorado,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001267?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001267_20...,"[{'chamber': 'Senate', 'startYear': 2009}]"
35,B001243,NaN,"Blackburn, Marsha",Republican,Tennessee,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001243?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001243_20...,"[{'chamber': 'House of Representatives', 'endY..."
36,B001236,NaN,"Boozman, John",Republican,Arkansas,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001236?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001236_20...,"[{'chamber': 'House of Representatives', 'endY..."
37,B001230,NaN,"Baldwin, Tammy",Democratic,Wisconsin,2024-03-10T12:42:12Z,https://api.congress.gov/v3/member/B001230?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001230_20...,"[{'chamber': 'House of Representatives', 'endY..."


In [10]:
replace_map = {'Republican': 'R',
               'Democratic': 'D',
               'Independent': 'I'}
members['partyletter'] = members['partyName'].replace(replace_map)
members

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item,partyletter
0,L000397,18.0,"Lofgren, Zoe",Democratic,California,2024-10-22T14:54:19Z,https://api.congress.gov/v3/member/L000397?for...,Image courtesy of the Member,https://www.congress.gov/img/member/671024d7ec...,"[{'chamber': 'House of Representatives', 'star...",D
1,L000604,4.0,"Lopez, Greg",Republican,Colorado,2024-10-10T21:25:00Z,https://api.congress.gov/v3/member/L000604?for...,Image courtesy of the Member,https://www.congress.gov/img/member/668e94fd65...,"[{'chamber': 'House of Representatives', 'star...",R
2,M001225,15.0,"Mullin, Kevin",Democratic,California,2024-10-10T21:22:27Z,https://api.congress.gov/v3/member/M001225?for...,Image courtesy of the Member,https://www.congress.gov/img/member/m001225_20...,"[{'chamber': 'House of Representatives', 'star...",D
3,M001229,10.0,"McIver, LaMonica",Democratic,New Jersey,2024-10-10T21:19:13Z,https://api.congress.gov/v3/member/M001229?for...,Image courtesy of the Member,https://www.congress.gov/img/member/66fd489d79...,"[{'chamber': 'House of Representatives', 'star...",D
4,H001097,NaN,"Helmy, George S.",Democratic,New Jersey,2024-10-10T12:42:18Z,https://api.congress.gov/v3/member/H001097?for...,Official U.S. Senate Photo,https://www.congress.gov/img/member/6705445b99...,"[{'chamber': 'Senate', 'startYear': 2024}]",D
...,...,...,...,...,...,...,...,...,...,...,...
34,B001267,NaN,"Bennet, Michael F.",Democratic,Colorado,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001267?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001267_20...,"[{'chamber': 'Senate', 'startYear': 2009}]",D
35,B001243,NaN,"Blackburn, Marsha",Republican,Tennessee,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001243?for...,Image courtesy of the Member,https://www.congress.gov/img/member/b001243_20...,"[{'chamber': 'House of Representatives', 'endY...",R
36,B001236,NaN,"Boozman, John",Republican,Arkansas,2024-03-10T12:42:13Z,https://api.congress.gov/v3/member/B001236?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001236_20...,"[{'chamber': 'House of Representatives', 'endY...",R
37,B001230,NaN,"Baldwin, Tammy",Democratic,Wisconsin,2024-03-10T12:42:12Z,https://api.congress.gov/v3/member/B001230?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/b001230_20...,"[{'chamber': 'House of Representatives', 'endY...",D


In [11]:
members['lastname'] = [x.split(',')[0] for x in members['name']]
members['firstname'] = [x.split(',')[1] for x in members['name']]
members['name2'] = [x + ' ' + y + ' (' + z + ')' 
                    for x, y, z in 
                    zip(members['firstname'], members['lastname'], members['partyletter'])]
members['name2']

0            Zoe Lofgren (D)
1             Greg Lopez (R)
2           Kevin Mullin (D)
3        LaMonica McIver (D)
4        George S. Helmy (D)
               ...          
34     Michael F. Bennet (D)
35      Marsha Blackburn (R)
36          John Boozman (R)
37         Tammy Baldwin (D)
38         Chris Stewart (R)
Name: name2, Length: 539, dtype: object

In [6]:
cands = pd.read_csv('data/cands22.txt', quotechar='|', header=None)
cands.columns = ['Cycle', 'FECCandID', 'CID','FirstLastP',
                 'Party','DistIDRunFor','DistIDCurr',
                 'CurrCand','CycleCand','CRPICO','RecipCode','NoPacs']
cands

,Cycle,FECCandID,CID,FirstLastP,Party,DistIDRunFor,DistIDCurr,CurrCand,CycleCand,CRPICO,RecipCode,NoPacs
0,2022,H0AK00105,N00039029,Thomas Lamb (3),3,AK01,,,,,3N,
1,2022,H0AL01055,N00044245,Jerry Carl (R),R,AL01,AL01,Y,Y,I,RW,
2,2022,H0AL01063,N00044288,Wes Lambert (R),R,AL01,,,,,RN,
3,2022,H0AL01097,N00044750,James Averhart (D),D,AL01,,,,,DN,
4,2022,H0AL02087,N00030768,Martha Roby (R),R,AL02,,,,,RN,
...,...,...,...,...,...,...,...,...,...,...,...,...
8923,2022,S8WI00224,N00041929,Leah Vukmir (R),R,WIS1,,,,,RN,
8924,2022,S8WV00119,N00041293,Paula Jean Swearengin (D),D,WVS2,,,,,DN,
8925,2022,S8WV00127,N00035531,Evan Jenkins (R),R,WVS1,,,,,RN,
8926,2022,S8WV00135,N00041474,Bo Copley (R),R,WVS1,,,,,RN,


In [ ]:
crosswalk = pd.merge(members, cands, 
                     left_on='name2',
                     right_on='FirstLastP',
                     how = 'outer',
                     indicator = 'matched',
                     validate='one_to_one')

In [3]:
goodbioguide = ct.get_bioguide(name='good', state='Virginia', district=5)
goodbioguide

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item
0,G000595,5.0,"good, bob",Republican,Virginia,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/G000595?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000595_20...,"[{'chamber': 'House of Representatives', 'star..."
1,G000280,5.0,"goode, virgil h., jr.",Republican,Virginia,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/G000280?for...,"Image, Congressional Pictorial Directory, 109th.",https://www.congress.gov/img/member/g000280_20...,"[{'chamber': 'House of Representatives', 'endY..."


In [4]:
goodid = goodbioguide.reset_index()['bioguideId'][0]
goodid

'G000595'

In [5]:
goodbills = ct.get_sponsoredlegislation(goodid)

In [ ]:
billurl = goodbills[0]['url']

In [7]:
ct.get_billdata(billurl)

{'bill': {'actions': {'count': 3,
   'url': 'https://api.congress.gov/v3/bill/118/hres/1509/actions?format=json'},
  'committees': {'count': 1,
   'url': 'https://api.congress.gov/v3/bill/118/hres/1509/committees?format=json'},
  'congress': 118,
  'cosponsors': {'count': 30,
   'countIncludingWithdrawnCosponsors': 30,
   'url': 'https://api.congress.gov/v3/bill/118/hres/1509/cosponsors?format=json'},
  'introducedDate': '2024-09-25',
  'latestAction': {'actionDate': '2024-09-25',
   'text': 'Referred to the House Committee on Energy and Commerce.'},
  'number': '1509',
  'originChamber': 'House',
  'originChamberCode': 'H',
  'sponsors': [{'bioguideId': 'G000595',
    'district': 5,
    'firstName': 'Bob',
    'fullName': 'Rep. Good, Bob [R-VA-5]',
    'isByRequest': 'N',
    'lastName': 'Good',
    'party': 'R',
    'state': 'VA',
    'url': 'https://api.congress.gov/v3/member/G000595?format=json'}],
  'textVersions': {'count': 1,
   'url': 'https://api.congress.gov/v3/bill/118/hres/

In [7]:
import requests
r = requests.get(goodbills[0]['url'],
                 params={'api_key': ct.congresskey})
texturl = json.loads(r.text)['bill']['textVersions']['url']
r = requests.get(texturl,
                 params={'api_key': ct.congresskey})
toscrape = json.loads(r.text)['textVersions'][0]['formats'][0]['url']

In [9]:
from bs4 import BeautifulSoup

r = requests.get(toscrape)
mysoup = BeautifulSoup(r.text, 'html.parser')
billtext =  mysoup.text

<html><body><pre>
[Congressional Bills 118th Congress]
[From the U.S. Government Publishing Office]
[H. Res. 1509 Introduced in House (IH)]

&lt;DOC&gt;






118th CONGRESS
  2d Session
H. RES. 1509

Supporting the designation of the week beginning November 11, 2024, as 
  ``National Pregnancy Center Week'' to recognize the vital role that 
community-supported pregnancy centers play in saving lives and serving 
        women and men faced with difficult pregnancy decisions.


_______________________________________________________________________


                    IN THE HOUSE OF REPRESENTATIVES

                           September 25, 2024

 Mr. Good of Virginia (for himself, Mr. Clyde, Mr. Mooney, Mr. Gosar, 
 Mr. Posey, Mr. Burgess, Mr. Bacon, Mr. Fleischmann, Mr. Grothman, Mr. 
Rose, Mr. Fulcher, Mr. Biggs, Mr. Higgins of Louisiana, Mr. Ogles, Mr. 
 Kelly of Pennsylvania, Mr. Lamborn, Mrs. Harshbarger, Mr. Norman, Mr. 
  Wilson of South Carolina, Mrs. Miller of Illinois, Mr. 

In [11]:
mysoup.find_all('pre')

[<pre>
 [Congressional Bills 118th Congress]
 [From the U.S. Government Publishing Office]
 [H. Res. 1509 Introduced in House (IH)]
 
 &lt;DOC&gt;
 
 
 
 
 
 
 118th CONGRESS
   2d Session
 H. RES. 1509
 
 Supporting the designation of the week beginning November 11, 2024, as 
   ``National Pregnancy Center Week'' to recognize the vital role that 
 community-supported pregnancy centers play in saving lives and serving 
         women and men faced with difficult pregnancy decisions.
 
 
 _______________________________________________________________________
 
 
                     IN THE HOUSE OF REPRESENTATIVES
 
                            September 25, 2024
 
  Mr. Good of Virginia (for himself, Mr. Clyde, Mr. Mooney, Mr. Gosar, 
  Mr. Posey, Mr. Burgess, Mr. Bacon, Mr. Fleischmann, Mr. Grothman, Mr. 
 Rose, Mr. Fulcher, Mr. Biggs, Mr. Higgins of Louisiana, Mr. Ogles, Mr. 
  Kelly of Pennsylvania, Mr. Lamborn, Mrs. Harshbarger, Mr. Norman, Mr. 
   Wilson of South Carolina, Mrs. Mi

In [6]:
import requests

# sponsored legislation
root = 'https://api.congress.gov/v3/'
endpoint = f'/member/{goodid}/sponsored-legislation'
headers = ct.make_headers()
params = {'api_key': ct.congresskey,
          'limit': 250}
r = requests.get(root + endpoint,
                 headers=headers,
                 params=params)
r

<Response [200]>

In [10]:
json.loads(r.text)['sponsoredLegislation'][2]

{'congress': 118,
 'introducedDate': '2024-05-23',
 'latestAction': {'actionDate': '2024-07-05',
  'text': 'Placed on the Union Calendar, Calendar No. 474.'},
 'number': '8534',
 'policyArea': {'name': None},
 'title': 'Protecting Student Athletes’ Economic Freedom Act of 2024',
 'type': 'HR',
 'url': 'https://api.congress.gov/v3/bill/118/hr/8534?format=json'}

In [3]:
useragent = ct.get_useragent()
useragent

'python-requests/2.32.3'

In [4]:
headers = ct.make_headers()
headers

{'User-Agent': 'python-requests/2.32.3', 'From': 'fvq3xv@virginia.edu'}

In [5]:
members_df = ct.get_bioguideIDs()

In [10]:
members_df

,bioguideId,district,name,partyName,state,updateDate,url,terms.item,depiction.attribution,depiction.imageUrl
0,M001229,10.0,"McIver, LaMonica",Democratic,New Jersey,2024-09-24T20:50:13Z,https://api.congress.gov/v3/member/M001229?for...,"[{'chamber': 'House of Representatives', 'star...",NaN,NaN
1,E000236,NaN,"Evans, Daniel J.",Republican,Washington,2024-09-23T14:08:49Z,https://api.congress.gov/v3/member/E000236?for...,"[{'chamber': 'Senate', 'endYear': 1989, 'start...","<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/e000236_20...
2,S000068,NaN,"Sasser, Jim",Democratic,Tennessee,2024-09-12T19:33:39Z,https://api.congress.gov/v3/member/S000068?for...,"[{'chamber': 'Senate', 'endYear': 1995, 'start...","<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/s000068_20...
3,H001097,NaN,"Helmy, George S.",Democratic,New Jersey,2024-09-12T12:42:19Z,https://api.congress.gov/v3/member/H001097?for...,"[{'chamber': 'Senate', 'startYear': 2024}]",NaN,NaN
4,R000599,25.0,"Ruiz, Raul",Democratic,California,2024-09-11T16:29:00Z,https://api.congress.gov/v3/member/R000599?for...,"[{'chamber': 'House of Representatives', 'star...",Image courtesy of the Member,https://www.congress.gov/img/member/66e1aec832...
...,...,...,...,...,...,...,...,...,...,...
19,F000229,6.0,"Flynt, John J., Jr.",Democratic,Georgia,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/F000229?for...,"[{'chamber': 'House of Representatives', 'endY...",Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/f000229_20...
20,F000218,7.0,"Flowers, Walter",Democratic,Alabama,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/F000218?for...,"[{'chamber': 'House of Representatives', 'endY...",Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/f000218_20...
21,F000215,1.0,"Florio, James J.",Democratic,New Jersey,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/F000215?for...,"[{'chamber': 'House of Representatives', 'endY...",Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/f000215_20...
22,F000209,11.0,"Flood, Daniel J.",Democratic,Pennsylvania,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/F000209?for...,"[{'chamber': 'House of Representatives', 'endY...",Collection of the U.S. House of Representatives,https://www.congress.gov/img/member/f000209_20...


In [7]:
ideo = ct.get_ideology()
members = ideo[['bioname', 'icpsr', 'party_code']]
members

,bioname,icpsr,party_code
0,"ROGERS, Mike Dennis",20301,200
1,"SEWELL, Terri",21102,100
2,"PALMER, Gary James",21500,200
3,"CARL, Jerry L.",22108,200
4,"MOORE, Barry",22140,200
...,...,...,...
443,"STEIL, Bryan",21970,200
444,"TIFFANY, Thomas P.",21989,200
445,"FITZGERALD, Scott",22115,200
446,"VAN ORDEN, Derrick",22370,200


In [8]:
votes = ct.get_votes()
votes

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,118,House,1,14854,6,100.0
1,118,House,1,14863,6,98.2
2,118,House,1,14873,1,100.0
3,118,House,1,15029,1,100.0
4,118,House,1,15433,1,100.0
...,...,...,...,...,...,...
486659,118,House,1118,31101,6,100.0
486660,118,House,1118,31102,9,99.6
486661,118,House,1118,39301,6,100.0
486662,118,House,1118,39307,6,100.0


In [ ]:
house = house[['rollnumber', 'icpsr', 'cast_code']]

In [ ]:
house_mat = pd.merge(house, house, on='rollnumber')
house_mat

In [ ]:
house_mat = pd.merge(house, house, on='rollnumber')
house_mat['agree'] = house_mat['cast_code_x'] == house_mat['cast_code_y']
house_mat = house_mat.groupby(['icpsr_x', 'icpsr_y'])['agree'].agg({'agree': 'mean'})
house_mat = house_mat.reset_index()

In [ ]:
agreement = pd.merge(house_mat, members,
                     left_on = 'icpsr_x',
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member1',
                              'party_code':'party1'}, axis=1)
agreement = agreement.drop(['icpsr_x'], axis=1)
agreement = pd.merge(agreement, members,
                     left_on = 'icpsr_y',
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member2',
                              'party_code':'party2'}, axis=1)
agreement = agreement[['member1', 'party1', 'member2', 'party2', 'agree']]
agreement


In [ ]:
agreement[agreement['member1'].str.contains('GOOD,')].sort_values('agree',  ascending=False).head(30)